## Sonu Giri
> contact: sonugiri1043@gmail.com

- DataSet: Tiny ImageNet
- Model: ResNet-50 ( Without Fully Connected layer )

Tiny ImageNet dataset consists of 200 categories and each category has 500 of 64x64 size images in training set.

## Load Tiny ImageNet DataSet

In [3]:
# load Google Driver
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
my_path = '/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/'

Mounted at /content/drive


In [4]:
!rm -rf '/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/__pycache__'
!ls -lrt '/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/'

total 242907
-rw------- 1 root root 248100043 Oct 18 07:54  tiny-imagenet-200.zip
-rw------- 1 root root      2132 Oct 18 08:35  dataAlbumentationLoad.py
-rw------- 1 root root      4849 Oct 18 08:35  dataTorchTransformLoad.py
-rw------- 1 root root      3051 Oct 18 08:35  DH_NET.py
-rw------- 1 root root      5882 Oct 18 08:35  GradCam.py
-rw------- 1 root root     31322 Oct 18 08:35  grid_image.png
-rw------- 1 root root      3657 Oct 18 08:35  Plots.py
-rw------- 1 root root      4077 Oct 18 08:36  QuizDNN.py
-rw------- 1 root root       575 Oct 18 08:36  README.md
-rw------- 1 root root      5380 Oct 18 08:36  Test_Train.py
-rw------- 1 root root      3694 Oct 18 08:36  VS_NET.py
-rw------- 1 root root      5554 Oct 18 08:37  Test_Train_OCP.py
drwx------ 2 root root      4096 Oct 18 08:37  outputs
drwx------ 2 root root      4096 Oct 18 10:42  tiny-imagenet-200
-rw------- 1 root root    427114 Oct 18 14:10  Tiny_Imagenet.ipynb
-rw------- 1 root root      4496 Oct 18 14:58  Resnet.p

In [5]:
# import

from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.functional as F
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau
import sys
import random
import matplotlib

In [6]:
sys.path.append('/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/')

In [7]:
# import for all the modular codes
import VS_NET
import Test_Train
import Test_Train_OCP
import Plots
import dataTorchTransformLoad as dtl
import Resnet
import GradCam
import Resnet
import dataAlbumentationLoad as dal
import GradCam
from GradCam import GradCAM, visualize_cam, save_misclassified, download_img_pil,pil_img_transform
from Plots import plot_misclassified,imshow
import DH_NET

In [23]:
my_path

'/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/'

In [9]:
from scipy.ndimage import imread

In [20]:
import zipfile

path_to_zip_file = "/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/tiny-imagenet-200.zip"
directory_to_extract_to = '/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/'
print("Extracting zip file: %s" % path_to_zip_file)
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)
print("Extracted at: %s" % directory_to_extract_to)

Extracting zip file: /content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/tiny-imagenet-200.zip
Extracted at: /content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet/


In [9]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

Cloning into 'IMagenet'...
remote: Enumerating objects: 120594, done.
remote: Total 120594 (delta 0), reused 0 (delta 0), pack-reused 120594
Receiving objects: 100% (120594/120594), 212.68 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (1115/1115), done.
Checking out files: 100% (120206/120206), done.
test  train  val  wnids.txt  words.txt


In [11]:
!pip uninstall scipy

!pip install scipy==1.1.0

Uninstalling scipy-1.1.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.1.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? y
  Successfully uninstalled scipy-1.1.0
  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


## Extract test and training data from Tiny ImageNet DataSet

In [8]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from tqdm import notebook
import random
import pdb


class ProcessTinyImageNet():

    path = '/content/drive/My Drive/Colab Notebooks/EVA 5/12.Yolo+Tinyimagenet//tiny-imagenet-200/'

    def __init__(self, classes_map):
        self.image = []
        self.target = []
        self.classes = classes_map
        self.update_train_val_data()

    @staticmethod
    def get_train_image_ids():
        """
            returns: ['n02321529',...]
        """
        path = ProcessTinyImageNet.path + "wnids.txt"
        train_image_ids = [line.strip() for line in open(path)]
        return train_image_ids

    @staticmethod
    def all_class_map():
        """
        eg:
        {'n02423022': 'label1'}
        """
        all_class_map = {}
        windis = ProcessTinyImageNet.get_train_image_ids()
        for line in open(ProcessTinyImageNet.path + "words.txt", 'r'):
            data = line.split('\t')[:2]
            n_id = data[0]
            class_name = data[1].strip()
            if n_id in windis:
              all_class_map[n_id] = class_name
        return all_class_map

    @staticmethod
    def get_image(image_path):
        img = Image.open(image_path)
        npimg = np.asarray(img)
        if len(npimg.shape) == 2:
            npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
        return npimg

    def update_train_data(self):
        """
        returns: {'train/n02124075/images/n02124075_10.JPEG': 'Egyptian cat'}
        """
        image_ids = ProcessTinyImageNet.get_train_image_ids()
        class_map = self.classes
        train_image_path = ProcessTinyImageNet.path + "train/{image_id}/images/{image_name}.JPEG"
        for image_id in notebook.tqdm(image_ids, desc='Loading train folder...'):
            for key_id in range(500):
                image_name = image_id + "_" + str(key_id)
                train_path = train_image_path.format(image_id=image_id, image_name=image_name)
                #npimg = ProcessTinyImageNet.get_image(train_path)
                train_label = class_map[image_id]
                self.image.append(train_path)
                self.target.append(train_label)

    def update_val_data(self):
        """
        returns: {'val/images/val_10.JPEG': 'swimming trunks, bathing trunks'}
        """
        path = ProcessTinyImageNet.path + "val/val_annotations.txt"
        class_map = self.classes
        val_image_path = ProcessTinyImageNet.path + "val/images/{image_name}"
        for line in notebook.tqdm(open(path, 'r'), desc='Loading val folder....'):
            data = line.split('\t')[:2]
            image_name = data[0].strip()
            image_id = data[1].strip()
            image_path = val_image_path.format(image_name=image_name)
            #npimg = ProcessTinyImageNet.get_image(image_path)
            val_label = class_map[image_id]
            self.image.append(image_path)
            self.target.append(val_label)

    def update_train_val_data(self):
        self.update_train_data()
        self.update_val_data()

## Shuffle training data

In [9]:
classes_map = ProcessTinyImageNet.all_class_map()
obj = ProcessTinyImageNet(classes_map)
#targets = torch.as_tensor(obj.target)
dataset = list(zip(obj.image, obj.target))
random.shuffle(dataset)
trainlen = int(len(dataset)*0.7)
classes = list(classes_map.values())
print(len(dataset), trainlen, len(classes))


110000 77000 200


In [10]:
import torch
from skimage import io
import numpy as np
from torch.utils.data import Dataset
from sklearn import preprocessing
from PIL import Image

class TinyImagenetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data, transform=None):
        """
        Args:
            data (string): zipped images and labels.
        """
        self.transform = transform
        self.images, self.labels = zip(*data)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = io.imread(self.images[idx], as_gray=False, pilmode="RGB")
        # image = Image.open(self.images)
        # npimg = np.asarray(image)
        # if (len(npimg.shape) == 2):
        #     npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
        le = preprocessing.LabelEncoder()
        targets = le.fit_transform(self.labels)
        self.labels = torch.as_tensor(targets)
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx] #label

In [11]:
import albumentations as A
class Albumentation:
  """
  Helper class to create test and train transforms using Albumentations
  """
  def __init__(self, transforms=[]):
    self.transforms = A.Compose(transforms)

  def __call__(self, img):
    img = np.array(img)
    return self.transforms(image=img)['image']

In [12]:
patch_size = 32
mean = (0.48043839, 0.44820218, 0.39760034)
std = (0.27698959, 0.26908774, 0.28216029)
custom_transforms = [
       A.PadIfNeeded(min_height=70, min_width=70),
       A.RandomCrop(64, 64, p=1.0),
       A.HorizontalFlip(p=0.25),
       A.Rotate(limit=15, p=0.25),
       A.Normalize(mean=mean, std=std),
       A.Cutout(num_holes=4, max_h_size=16, max_w_size=16),
    ]

train_transforms = Albumentation(custom_transforms)
test_transforms = Albumentation(
    [
     A.Normalize(mean=mean, std=std),
     ])

In [13]:

# train_dataset = TorchVisionDataSet(train, transform=train_transforms)
# test_dataset = TorchVisionDataSet(test, transform=test_transforms)
train_dataset = TinyImagenetDataset(dataset[:trainlen], transform=train_transforms)
test_dataset = TinyImagenetDataset(dataset[trainlen:], transform=test_transforms)

In [44]:

class DataLoader(object):

    def __init__(self, train, test, batch_size=128):
        self.train_loader, self.test_loader = self.data_loader(train, test, batch_size)

    def data_loader(self, train, test, batch_size=128,  num_workers=4):
        SEED = 1
        # CUDA?
        cuda = torch.cuda.is_available()
        print("CUDA Available?", cuda)

        # For reproducibility
        torch.manual_seed(SEED)

        if cuda:
            torch.cuda.manual_seed(SEED)

        # dataloader arguments - something you'll fetch these from cmdprmt
        dataloader_args = dict(shuffle=True, batch_size=batch_size, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

        # train dataloader
        train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

        # test dataloader
        test_loader = torch.utils.data.DataLoader(test, **dataloader_args)
        return train_loader, test_loader

In [45]:

data_loader_obj = DataLoader(train_dataset, test_dataset, batch_size=256)
train_loader = data_loader_obj.train_loader
test_loader = data_loader_obj.test_loader

CUDA Available? True


In [43]:
train_loader.shape

AttributeError: ignored

In [15]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Resnet.ResNet18().to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [30]:

#initializers 
dropout_value = 0.05
num_splits=2
start_epoch=0
EPOCHS = 24
EPOCHS1 = 50
input_ch=3
best_acc = 60

metric_values = {}
models={}

batch_size_GPU=256
batch_size_CPU=16

model_save_path = my_path+"/best_model.pt"


In [18]:
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.05)

In [48]:
exp_name = 'Best Learning Rate'

trainer = Test_Train_OCP.Trainer(model,device,train_loader,test_loader,optimizer,start_epoch,EPOCHS,scheduler,model_save_path,best_acc)
train_metric, test_metric,learning_rate = trainer.train(l1_lambda=0, l2_lambda=0)

# save the metrics in dictionary
metric_values[exp_name] = (train_metric, test_metric)









  0%|          | 0/301 [00:00<?, ?it/s]

EPOCH: 0
learning rate  0.01


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0d5e5bb278>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0d5e5bb278>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0d5e5bb278>>
Traceback (most recent call last):
Traceback (most recent call last):
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0d5e5bb278>>
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1101, in __del__
Traceback (most recent call last):
    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/datal

RuntimeError: ignored

##Build Resnet Model

In [ ]:
import six
from keras.models import Model
from keras.layers import ( Input, Activation, Dense, Flatten )
from keras.layers.convolutional import ( Conv2D, MaxPooling2D, AveragePooling2D )
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np

def _bn_relu(input):
    """Helper to build a BN -> relu block"""
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)

def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block"""
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f

def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f

def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum" """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks."""
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input
    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4 """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1,1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras 'Model'.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
 
        out = Conv2D(filters=num_outputs, kernel_size=(1, 1),
                              strides=(1,1),
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4),
                              activation="softmax")(pool2)
        flatten1 = Flatten()(out)
        model = Model(inputs=input, outputs=flatten1)
        return model

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

Using TensorFlow backend.


In [ ]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet50_tiny_ImageNet.csv')

batch_size = 500
nb_classes = 200
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 64, 64
# The images are RGB
img_channels = 3

# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

model = ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [ ]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
          featurewise_center=False,           # set input mean to 0 over the dataset
          samplewise_center=False,            # set each sample mean to 0
          featurewise_std_normalization=False,# divide inputs by std of the dataset
          samplewise_std_normalization=False, # divide each input by its std
          zca_whitening=False,                # apply ZCA whitening
          rotation_range=0,                   # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,              # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,             # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,               # randomly flip images
          vertical_flip=False )               # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit( X_train )

Using real-time data augmentation.


In [ ]:
nb_epoch = 10 # epoch 1-10
# Fit the model on the batches generated by datagen.flow().
model.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),
                     steps_per_epoch=X_train.shape[0] // batch_size,
                     validation_data=(X_test, Y_test),
                     epochs=nb_epoch, verbose=1, max_q_size=100,
                     callbacks=[lr_reducer, early_stopper, csv_logger] )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  import sys


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
200/200 [==============================] - 313s 2s/step - loss: 7.8622 - acc: 0.1091 - val_loss: 6.5234 - val_acc: 0.1111
Epoch 2/10
200/200 [==============================] - 284s 1s/step - loss: 5.4329 - acc: 0.2009 - val_loss: 5.2347 - val_acc: 0.1820
Epoch 3/10
200/200 [==============================] - 282s 1s/step - loss: 4.6477 - acc: 0.2485 - val_loss: 4.6894 - val_acc: 0.2170
Epoch 4/10
200/200 [==============================] - 282s 1s/step - loss: 4.2260 - acc: 0.2819 - val_loss: 4.4114 - val_acc: 0.2402
Epoch 5/10
200/200 [==============================] - 281s 1s/step - loss: 3.9444 - acc: 0.3107 - val_loss: 4.2279 - val_acc: 0.2517
Epoch 6/10
200/200 [==============================] - 281s 1s/step - loss: 3.7360 - acc: 0.3331 - val_loss: 4.0793 - val_acc: 0.2785
Epoch 7/10
200/200 [==============================] - 283s 1s/step - loss: 3

In [ ]:
epochs_passed = 10
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [ ]:
from keras.models import load_model

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 10 # epoch 11-20
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/10
200/200 [==============================] - 311s 2s/step - loss: 3.2031 - acc: 0.4109 - val_loss: 3.6135 - val_acc: 0.3344
Epoch 2/10
200/200 [==============================] - 284s 1s/step - loss: 3.1008 - acc: 0.4286 - val_loss: 3.5681 - val_acc: 0.3448
Epoch 3/10
200/200 [==============================] - 283s 1s/step - loss: 3.0402 - acc: 0.4400 - val_loss: 3.5631 - val_acc: 0.3517
Epoch 4/10
200/200 [==============================] - 286s 1s/step - loss: 2.9875 - acc: 0.4492 - val_loss: 3.5427 - val_acc: 0.3522
Epoch 5/10
200/200 [==============================] - 284s 1s/step - loss: 2.9247 - acc: 0.4614 - val_loss: 3.5898 - val_acc: 0.3516
Epoch 6/10
200/200 [==============================] - 283s 1s/step - loss: 2.8781 - acc: 0.4735 - val_loss: 3.6709 - val_acc: 0.3428
Epoch 7/10
200/200 [==============================] - 283s 1s/step - loss: 2.8397 - acc: 0.4792 - val_loss: 3.3663 - val_acc: 0.3853
Epoch 8/10
200/200 [==============================] - 282s 1s/step - 

In [ ]:
epochs_passed = 20
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [ ]:
from keras.models import load_model
epochs_passed = 20

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 10 # epoch 21-30
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/10
200/200 [==============================] - 313s 2s/step - loss: 2.7222 - acc: 0.5083 - val_loss: 3.4446 - val_acc: 0.3883
Epoch 2/10
200/200 [==============================] - 280s 1s/step - loss: 2.6477 - acc: 0.5233 - val_loss: 3.5928 - val_acc: 0.3657
Epoch 3/10
200/200 [==============================] - 281s 1s/step - loss: 2.6144 - acc: 0.5319 - val_loss: 3.4228 - val_acc: 0.3854
Epoch 4/10
200/200 [==============================] - 278s 1s/step - loss: 2.5891 - acc: 0.5370 - val_loss: 3.4009 - val_acc: 0.4019
Epoch 5/10
200/200 [==============================] - 280s 1s/step - loss: 2.5640 - acc: 0.5431 - val_loss: 3.4203 - val_acc: 0.3932
Epoch 6/10
200/200 [==============================] - 281s 1s/step - loss: 2.5398 - acc: 0.5482 - val_loss: 3.5287 - val_acc: 0.3887
Epoch 7/10
200/200 [==============================] - 280s 1s/step - loss: 2.5110 - acc: 0.5564 - val_loss: 3.5288 - val_acc: 0.3833
Epoch 8/10
200/200 [==============================] - 280s 1s/step - 

In [ ]:
epochs_passed = 30
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [ ]:
from keras.models import load_model
epochs_passed = 30

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 20 # epoch 31-50
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=20, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/20
200/200 [==============================] - 312s 2s/step - loss: 2.4577 - acc: 0.5752 - val_loss: 3.3310 - val_acc: 0.4151
Epoch 2/20
200/200 [==============================] - 280s 1s/step - loss: 2.3981 - acc: 0.5879 - val_loss: 3.4938 - val_acc: 0.4039
Epoch 3/20
200/200 [==============================] - 278s 1s/step - loss: 2.3730 - acc: 0.5941 - val_loss: 3.5969 - val_acc: 0.3922
Epoch 4/20
200/200 [==============================] - 278s 1s/step - loss: 2.3490 - acc: 0.6011 - val_loss: 3.6004 - val_acc: 0.3967
Epoch 5/20
200/200 [==============================] - 280s 1s/step - loss: 2.3398 - acc: 0.6040 - val_loss: 3.4154 - val_acc: 0.4206
Epoch 6/20
200/200 [==============================] - 279s 1s/step - loss: 2.3191 - acc: 0.6106 - val_loss: 3.3694 - val_acc: 0.4399
Epoch 7/20
200/200 [==============================] - 280s 1s/step - loss: 1.9283 - acc: 0.7093 - val_loss: 2.9221 - val_acc: 0.5035
Epoch 8/20
200/200 [==============================] - 277s 1s/step - 

In [ ]:
epochs_passed = 47
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )
print("Model and weights after %s epochs have been saved" % epochs_passed )

Model and weights after 47 epochs have been saved


In [ ]:
nb_epoch = 13 # epoch 47-60
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=13, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  import sys


Epoch 1/13
200/200 [==============================] - 232s 1s/step - loss: 1.1088 - acc: 0.9068 - val_loss: 2.9980 - val_acc: 0.5161
Epoch 2/13
200/200 [==============================] - 232s 1s/step - loss: 1.0894 - acc: 0.9118 - val_loss: 3.0060 - val_acc: 0.5143
Epoch 3/13
200/200 [==============================] - 233s 1s/step - loss: 1.0760 - acc: 0.9154 - val_loss: 3.0285 - val_acc: 0.5154
Epoch 4/13
200/200 [==============================] - 233s 1s/step - loss: 1.0610 - acc: 0.9191 - val_loss: 3.0449 - val_acc: 0.5137
Epoch 5/13
200/200 [==============================] - 233s 1s/step - loss: 1.0486 - acc: 0.9216 - val_loss: 3.0565 - val_acc: 0.5128
Epoch 6/13
200/200 [==============================] - 233s 1s/step - loss: 1.0363 - acc: 0.9252 - val_loss: 3.0628 - val_acc: 0.5115
Epoch 7/13
200/200 [==============================] - 233s 1s/step - loss: 1.0210 - acc: 0.9302 - val_loss: 3.0554 - val_acc: 0.5146
Epoch 8/13
200/200 [==============================] - 234s 1s/step - 

In [ ]:
epochs_passed = 60
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )
print("Model and weights after %s epochs have been saved" % epochs_passed )

Model and weights after 60 epochs have been saved
